In [4]:
# !pip install dash
!pip install dash-bootstrap-components


Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd 
import numpy as np 
from datetime import date, datetime, timedelta

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from bs4 import BeautifulSoup

import dash
import dash_bootstrap_components as dbc
from dash import html


import gdelt # for gdelt searchs

here


In [15]:
gdv2 = gdelt.gdelt(version=2) # Instantiate the GDELT object for searches
current_date = date.today().strftime('%Y %b %d') # Get the current date formatted as 'YYYY MMM DD'
print(current_date) # Print the current date

# current_gkg = gdv2.Search(date=['2024 Oct 08', current_date], table='gkg', normcols = True, coverage=True) # Search most recent GKG records from October 8, 2024 to today
# current_gkg = gdv2.Search(date=[current_date], table='gkg', normcols = True, coverage=True) # Search most recent GKG records from today.
current_gkg = gdv2.Search(date=['2024 Oct 21'], table='gkg',
                            normcols = True, coverage=True
                            # translation=True
                            ) # Search most recent GKG records from today.
print("The number of GKG records is", len(current_gkg)) # number of GKG records returned

2024 Nov 13
The number of GKG records is 140870


In [16]:
gkg = current_gkg.copy() # Copy the GKG records to a new DataFrame

In [17]:
gkg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140870 entries, 0 to 140869
Data columns (total 27 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   gkgrecordid                 140870 non-null  object 
 1   date                        140870 non-null  int64  
 2   sourcecollectionidentifier  140870 non-null  int64  
 3   sourcecommonname            140870 non-null  object 
 4   documentidentifier          140870 non-null  object 
 5   counts                      16673 non-null   object 
 6   v2counts                    16673 non-null   object 
 7   themes                      126415 non-null  object 
 8   v2themes                    126415 non-null  object 
 9   locations                   103964 non-null  object 
 10  v2locations                 103821 non-null  object 
 11  persons                     109143 non-null  object 
 12  v2persons                   107823 non-null  object 
 13  organizations 

In [18]:
# subset gkg where v2persons contains Eiichiro Oda Shueisha or One Piece
gkg_op = gkg[gkg['v2persons'].str.contains('Eiichiro Oda|Shueisha|One Piece', case=False, na=False)].copy()
gkg_op['date'] = pd.to_datetime(gkg_op['date'], format='%Y%m%d%H%M%S')
gkg_op

,gkgrecordid,date,sourcecollectionidentifier,sourcecommonname,documentidentifier,counts,v2counts,themes,v2themes,locations,...,gcam,sharingimage,relatedimages,socialimageembeds,socialvideoembeds,quotations,allnames,amounts,translationinfo,extras
219,20241021000000-219,2024-10-21 00:00:00,1,gamerant.com,https://gamerant.com/one-piece-chapter-1130-od...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_PRINCE;PIRACY;TAX_FNCACT...,"WB_1979_NATURAL_RESOURCE_MANAGEMENT,7018;WB_43...",NaN,...,"wc:1313,c1.1:3,c1.4:1,c12.1:111,c12.10:133,c12...",https://static0.gamerantimages.com/wordpress/w...,https://static0.gamerantimages.com/wordpress/w...,NaN,https://youtube.com/channel/UCkZjsmAQnXfS-_5lw...,NaN,"Piece Chapter,33;Accursed Prince,77;Straw Hat ...","2,Major Projects During The,532;2,major projec...",NaN,<PAGE_LINKS>https://gamerant.com/one-piece-113...
4492,20241021010000-585,2024-10-21 01:00:00,1,gamerant.com,https://gamerant.com/hunter-x-hunter-404-spoil...,NaN,NaN,ARMEDCONFLICT;EPU_CATS_NATIONAL_SECURITY;TAX_F...,"KILL,1931;ASSASSINATION,3335;CRISISLEX_O01_WEA...",NaN,...,"wc:1270,c1.3:1,c12.1:117,c12.10:189,c12.12:62,...",https://static0.gamerantimages.com/wordpress/w...,https://static0.gamerantimages.com/wordpress/w...,NaN,https://youtube.com/channel/UCkZjsmAQnXfS-_5lw...,NaN,"Succession War,342;Succession War,565;Longhi N...","404,will be an intense,288;26,degrees #x2026,1...",NaN,<PAGE_LINKS>https://gamerant.com/hunter-x-hunt...
8604,20241021021500-531,2024-10-21 02:15:00,1,screenrant.com,https://screenrant.com/luffy-stronger-elbaf-on...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_CAPTAIN;,"TAX_FNCACT_CAPTAIN,247;TAX_FNCACT_CAPTAIN,941;",NaN,...,"wc:609,c1.1:1,c12.1:44,c12.10:77,c12.12:26,c12...",https://static1.srcdn.com/wordpress/wp-content...,https://static1.srcdn.com/wordpress/wp-content...,NaN,https://youtube.com/user/screenrant?sub_confir...,2312|45||It was built to prevent giants from e...,"Straw Hat,34;One Piece,72;Straw Hat Captain,23...","5,Straw Hats woke up,525;2,being some of histo...",NaN,<PAGE_LINKS>https://screenrant.com/one-piece-g...
13753,20241021040000-128,2024-10-21 04:00:00,1,comicbook.com,https://comicbook.com/anime/news/dragon-ball-g...,NaN,NaN,TAX_ETHNICITY;TAX_ETHNICITY_BLACK;ARMEDCONFLIC...,"TAX_ETHNICITY_BLACK,38;TAX_ETHNICITY_BLACK,165...",NaN,...,"wc:614,c1.3:2,c12.1:47,c12.10:76,c12.12:29,c12...",https://comicbook.com/wp-content/uploads/sites...,NaN,NaN,https://youtube.com/user/ComicBookCom/playlists;,NaN,"Dragon Ball,12;Gohan Black,39;Goku Black,173;D...",NaN,NaN,<PAGE_LINKS>https://comicbook.com/anime/news/d...
122397,20241021211500-2010,2024-10-21 21:15:00,1,screenrant.com,https://screenrant.com/one-piece-fish-man-isla...,NaN,NaN,NaN,NaN,NaN,...,"wc:331,c12.1:23,c12.10:22,c12.12:8,c12.13:3,c1...",https://static1.srcdn.com/wordpress/wp-content...,NaN,NaN,https://youtube.com/user/screenrant?sub_confir...,NaN,"One Piece,10;Fishman Island Saga,58;One Piece,...","2,being some of history,1535;50,video games ma...",NaN,<PAGE_LINKS>https://screenrant.com/db/tv-show/...
127452,20241021220000-973,2024-10-21 22:00:00,1,gamerant.com,https://gamerant.com/one-piece-oda-reveals-lok...,NaN,NaN,TAX_FNCACT;TAX_FNCACT_FATHER;ARREST;PIRACY;TAX...,"PIRACY,638;PIRACY,4983;PIRACY,5022;TAX_FNCACT_...",1#Reunion#RE#RE#-21.1#55.6#RE,...,"wc:1245,c1.1:1,c1.2:2,c1.4:4,c12.1:147,c12.10:...",https://static0.gamerantimages.com/wordpress/w...,https://static0.gamerantimages.com/wordpress/w...,NaN,https://youtube.com/channel/UCkZjsmAQnXfS-_5lw...,NaN,"Devil Fruit,90;One Piece,189;One Piece,247;Pie...",NaN,NaN,<PAGE_LINKS>https://gamerant.com/one-piece-cha...
128638,20241021221500-99,2024-10-21 22:15:00,1,screenrant.com,https://screenrant.com/one-piece-fan-letter-pr...,NaN,NaN,NaN,NaN,NaN,...,"wc:309,c1.1:7,c1.2:1,c12.1:26,c12.10:13,c12.12...",https://static1.srcdn.com/wordpress/wp-content...,NaN,NaN,https://youtube.com/user/screenrant?sub_confir...,NaN,"One Piece,10;Fan Letter,90;One-Punch Man,171;M...","1000000,of fans worldwide,268;2,being some of ...",NaN

In [19]:
gkg_op['documentidentifier'].value_counts() # Count the number of records for each document identifier

documentidentifier
https://gamerant.com/one-piece-chapter-1130-oda-finally-introduces-loki/           1
https://gamerant.com/hunter-x-hunter-404-spoilers-release-date-leaks/              1
https://screenrant.com/luffy-stronger-elbaf-one-piece-giants-straw-hats/           1
https://comicbook.com/anime/news/dragon-ball-gohan-black-goku-black-changes/       1
https://screenrant.com/one-piece-fish-man-island-remaster-new-preview/             1
https://gamerant.com/one-piece-oda-reveals-loki-legendary-strength/                1
https://screenrant.com/one-piece-fan-letter-praise-one-punch-man-artist-murata/    1
https://gamerant.com/one-piece-chapter-1131-delayed-new-release-date-announced/    1
Name: count, dtype: int64

We see that every document in the gkg network is unique. Let us assume it is the case the `sourcecommonname` is the publisher of each unique document. 

This means we can track the volume published by a source in time.

# Get Article Title Using Beautiful Soup

In [20]:
def fetch_article_title(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    session = requests.Session()
    retry = Retry(connect=2, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Try different potential title locations
        title_tag = soup.find('h1', class_='article-header-title')
        if not title_tag:
            title_tag = soup.find('h1', class_='post-title')
        
        # If still not found, look for any h1 tag
        if not title_tag:
            title_tag = soup.find('h1')
        
        if title_tag:
            return title_tag.get_text(strip=True)
        return "Title not found"
    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

# Fetch titles for all URLs
gkg_op['title'] = gkg_op['documentidentifier'].apply(fetch_article_title)

In [21]:
gkg_op['title'].value_counts()

title
One Piece Chapter 1130: Oda Finally Introduces Loki                                 1
Hunter x Hunter 404: What To Expect From The Chapter                                1
Luffy's Already Proved He's Far Stronger Than Elbaf's Strongest One Piece Giants    1
Dragon Ball Fixes One of Goku Black’s Worst Traits With Gohan Black                 1
One Piece's Fish-Man Island Remaster Gets First Look In Exciting New Preview        1
One Piece: Oda Reveals Loki's Legendary Strength                                    1
New One Piece Episode Earns Glowing Praise From Acclaimed One-Punch Man Artist      1
One Piece Chapter 1131 Delayed, New Release Date Announced                          1
Name: count, dtype: int64

# One Piece News Dashboard

In [22]:
import webbrowser
from webbrowser import BackgroundBrowser

# Define the path to Microsoft Edge executable
# Note: Update the path to the actual location of your Microsoft Edge executable
edge_path = "C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe"

# Register Microsoft Edge as a browser type
# The 'BackgroundBrowser' class is used to pass the path of the browser executable
class Edge(BackgroundBrowser):
    def __init__(self, path):
        super().__init__(path)


# Register Microsoft Edge as a browser type
webbrowser.register('edge', None, webbrowser.BackgroundBrowser(edge_path))

In [23]:
# Create the Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# Create the image grid with titles
image_grid = html.Div(
    [
        html.Div(
            [
                html.A(
                    [
                        html.Img(src=row['sharingimage'], className="masonry-item"),
                        html.Div(row['title'], className="image-title")
                    ],
                    href=row['documentidentifier'],
                    target="_blank"
                )
            ],
            className="masonry-item-wrapper"
        )
        for _, row in gkg_op.iterrows()
    ],
    className="masonry"
)

app.layout = dbc.Container(
    [
        html.H1("One Piece Image Grid"),
        image_grid
    ],
    fluid=True,
    style={"padding": "10px"}
)

if __name__ == '__main__':
    import webbrowser
    url = "http://127.0.0.1:50779/"
    webbrowser.get('edge').open_new(url)
    app.run_server(debug=True, port=50779)